# Using LLMs in Humanities Research via API

## Session 3 16.00-17.30 - Intermediate LLM Usage and Advanced API Features

Additionally, we will examine challenges associated with historical digitized texts, including optical character recognition (OCR) errors, which may affect compatibility with language models. Participants will gain insights into how these models can be leveraged for error correction and translation, enhancing the usability of imperfect textual data.

## Session Outline

- **Advanced API Features**: Exploring advanced features of the OpenRouter API, including model selection, temperature settings, and response formatting.
- **Handling OCR Errors**: Discussing the challenges of working with historical digitized texts, including OCR errors, and how LLMs can assist in correcting these errors.
- **Practical Exercises**: Hands-on exercises to apply the concepts learned, including making API calls with advanced parameters and processing responses.

I'll add the corpus loading and verification code for Rigasche Zeitung to session 3, following the same structure as session 2 but adapted for the historical German newspaper corpus.




## BSSDH 2025 Workshop Data - Rigasche Zeitung Corpus

Before we dive into advanced LLM features and OCR error correction, let's load and examine our corpus of historical documents. For this session, we'll be working with the **Rigasche Zeitung** corpus, which presents unique challenges due to its historical nature and OCR quality.

Data for workshops in [Baltic Summer School of Digital Humanities 2025](https://www.digitalhumanities.lv/bssdh/2025/about/)

**Repository:** https://github.com/LNB-DH/BSSDH_2025_workshop_data

## CORPUS OVERVIEW - RIGASCHE ZEITUNG

### 1. SOURCE MATERIAL

| Periodical | Details |
|------------|---------|
| **"Rigasche Zeitung" (RZei) (1918–1919)** | - **Data file:** `Rigasche_Zeitung_1918_1919.zip`<br>- **Download Rigasche Zeitung:** https://github.com/LNB-DH/BSSDH_2025_workshop_data/raw/main/data/Rigasche_Zeitung_1918_1919.zip<br>- Morning newspaper, intermittently published from 1778 to 1919 in Riga.<br>- **Language:** German (Fraktur script)<br>- Once the most popular morning paper in the Baltic provinces of the Russian Empire.<br>- Covered general political and economic news in Riga, the Baltics, the Russian Empire, and internationally.<br>- **Historical context:** World War I, Latvian War of Independence.<br>- Link: https://periodika.lv/#periodicalMeta:234;-1<br>- More info: https://enciklopedija.lv/skirklis/163962 |

### 2. CORPUS CHARACTERISTICS

| Metric | RZei (1918-1919) |
|--------|------------------|
| **Token Count (words)** | 5.37 million |
| **Issue Count** | 359 issues |
| **Segment (Article ⟷ File) Count** | 4,597 |
| **Language** | German |
| **Script** | Fraktur |
| **OCR Quality** | Lower than modern texts (historical challenges) |

**Filename Structure:**
Format: `[periodical][year][volume#][issue#]_[page#]_[plaintext]_[segment#]`

Example: `rzei1918s01n001_001_plaintext_s01.txt`
         → 1st segment from RZei, Issue 1, 1918, page 1.

### 3. HISTORICAL CONTEXT & CHALLENGES

#### **Why Rigasche Zeitung is Perfect for OCR Error Studies:**

1. **Historical Script:** Fraktur typeface presents unique OCR challenges
2. **Wartime Period:** 1918-1919 covers end of WWI and Latvian independence
3. **Print Quality:** Historical printing technology affects digitization quality
4. **Language Complexity:** Early 20th century German with period-specific terminology
5. **Physical Degradation:** Age-related paper deterioration impacts OCR accuracy

#### **Research Applications:**
- **OCR Error Correction:** Testing LLM capabilities on historical text
- **Historical Event Analysis:** WWI aftermath, Latvian independence movement
- **Language Evolution:** German language usage in the Baltic region
- **Cross-cultural Studies:** German-language perspective on Baltic events
- **Translation Challenges:** Historical German to modern languages

### 4. METHODOLOGY

| Step | Description |
|------|-------------|
| **4.1. Source Access** | Digitised issues obtained from the National Library of Latvia (https://periodika.lv/) |
| **4.2. Processing & OCR** | CCS docWORKS & ABBYY FineReader 9.0<br>- **Note:** RZei has lower OCR quality than modern texts<br>- No further data cleaning/normalization (preserves authentic OCR errors) |
| **4.3. Metadata Added** | Fields: title, author, uri<br>- **Author info:** 325 cases (7.05%)<br>- **Title availability:** 99.15%<br>- **URI coverage:** 100%<br>- URIs point to LNB DOM system |

**💡 Important Note:** The deliberate preservation of OCR errors in this corpus makes it ideal for testing LLM-based error correction techniques, which we'll explore in this session.


## Extracting Historical Documents

Working with historical corpora requires careful handling of data extraction and validation. Unlike modern digital texts, historical documents present unique challenges that we need to account for in our workflow.

### Additional Considerations for Historical Documents

#### **Data Integrity Concerns:**
* **Encoding Issues:** Historical texts may contain unusual characters or encoding problems
* **File Structure:** Complex directory hierarchies may reflect archival organization
* **Metadata Preservation:** Historical context information must be maintained
* **Version Control:** Track which OCR version or processing method was used

#### **Technical Considerations:**
* **Storage Location:** Consistent paths for local and cloud environments (Google Colab)
* **File Naming:** Preserve original archival naming conventions while ensuring accessibility
* **Error Handling:** Graceful handling of corrupted or incomplete files
* **Validation:** Verify extracted content matches expected corpus characteristics

#### **Research Workflow Integration:**
* **Reproducibility:** Document exact extraction procedures for research replication
* **Scalability:** Prepare for processing large numbers of historical documents
* **Compatibility:** Ensure extracted data works with downstream LLM processing
* **Backup Strategy:** Maintain original data integrity while allowing experimentation

### Extracting Rigasche Zeitung Corpus

For this session, we'll extract the **Rigasche Zeitung (RZei)** corpus, which contains 4,597 text segments from 359 newspaper issues. This corpus is particularly valuable for studying OCR error patterns and testing correction strategies.

We'll use a robust Python function that handles:
- **Secure download** from the GitHub repository
- **Automatic extraction** to a standardized directory structure
- **Error reporting** for troubleshooting
- **Performance monitoring** to track processing time

The function will download the zip file, extract it to an appropriate location, and return information about the extracted files for verification.

In [3]:

url = "https://github.com/LNB-DH/BSSDH_2025_workshop_data/raw/main/data/Rigasche_Zeitung_1918_1919.zip"
print("Will extract Rigasche Zeitung historical corpus from", url)

# Define a function to download and extract the zip file, making it reusable for other corpora
# We set default values for optional arguments to make the function flexible
def extract_zip(url, output_dir="data", verbose=False):
    """
    Download and extract a zip file from a URL.
    
    Args:
        url (str): URL to download the zip file from
        output_dir (str): Directory to extract files to (default: "data")
        verbose (bool): Whether to print detailed progress information
    
    Returns:
        bool: True if successful, False otherwise
    """
    # Import required libraries (keeping script self-contained)
    import requests
    from zipfile import ZipFile
    from io import BytesIO
    from datetime import datetime

    try:
        if verbose:
            download_start = datetime.now()
            print(f"🔄 Starting download at {download_start.strftime('%Y-%m-%d %H:%M:%S.%f')}")
            print(f"📥 Downloading from: {url}")
        
        # Download the file
        response = requests.get(url)
        
        if verbose:
            download_finish = datetime.now()
            download_duration = download_finish - download_start
            print(f"✅ Download completed at {download_finish.strftime('%Y-%m-%d %H:%M:%S.%f')}")
            print(f"⏱️  Download duration: {download_duration.total_seconds():.2f} seconds")
            print(f"📊 Downloaded {len(response.content):,} bytes")
        
        # Check if download was successful
        if response.status_code == 200:
            if verbose:
                extract_start = datetime.now()
                print(f"📂 Starting extraction to '{output_dir}' at {extract_start.strftime('%Y-%m-%d %H:%M:%S.%f')}")
            
            # Extract the zip file
            with ZipFile(BytesIO(response.content)) as zf:
                zf.extractall(output_dir)
            
            if verbose:
                extract_end = datetime.now()
                extract_duration = extract_end - extract_start
                total_duration = extract_end - download_start
                print(f"✅ Extraction completed at {extract_end.strftime('%Y-%m-%d %H:%M:%S.%f')}")
                print(f"⏱️  Extraction duration: {extract_duration.total_seconds():.2f} seconds")
                print(f"🎯 Total process duration: {total_duration.total_seconds():.2f} seconds")
                print(f"📁 Files extracted to: {output_dir}")
            
            return True
        else:
            print(f"❌ Failed to download data. HTTP status code: {response.status_code}")
            return False
            
    except Exception as e:
        print(f"❌ Error during download/extraction: {str(e)}")
        return False

# Extract the Rigasche Zeitung corpus
print("🚀 Starting Rigasche Zeitung corpus extraction...")
success = extract_zip(url, verbose=True)

if success:
    print("\n🎉 Rigasche Zeitung corpus successfully extracted!")
    print("📚 Ready to analyze historical German newspaper content from 1918-1919")
else:
    print("\n⚠️  Extraction failed. Please check your internet connection and try again.")

Will extract Rigasche Zeitung historical corpus from https://github.com/LNB-DH/BSSDH_2025_workshop_data/raw/main/data/Rigasche_Zeitung_1918_1919.zip
🚀 Starting Rigasche Zeitung corpus extraction...
🔄 Starting download at 2025-08-01 13:13:26.133700
📥 Downloading from: https://github.com/LNB-DH/BSSDH_2025_workshop_data/raw/main/data/Rigasche_Zeitung_1918_1919.zip
✅ Download completed at 2025-08-01 13:13:28.131537
⏱️  Download duration: 2.00 seconds
📊 Downloaded 16,776,843 bytes
📂 Starting extraction to 'data' at 2025-08-01 13:13:28.132537
✅ Extraction completed at 2025-08-01 13:13:51.411734
⏱️  Extraction duration: 23.28 seconds
🎯 Total process duration: 25.28 seconds
📁 Files extracted to: data

🎉 Rigasche Zeitung corpus successfully extracted!
📚 Ready to analyze historical German newspaper content from 1918-1919



### Verifying Extracted Historical Documents

After extracting historical corpora, it's crucial to verify that the extraction process completed successfully and that we have access to the expected files. This verification step is particularly important for historical documents where:

- **File integrity** may be affected by long-term digital preservation processes
- **Complex directory structures** might reflect archival organization systems
- **Encoding issues** could affect file accessibility
- **Large file counts** require systematic verification approaches

### Verification Process

We'll use Python's `pathlib` module to systematically check:
1. **Directory existence** and accessibility
2. **File count** and structure validation
3. **File naming patterns** to ensure they match expected conventions
4. **Initial content sampling** to verify readability

This verification step helps us catch any issues early in our workflow, before we begin the more computationally expensive LLM processing steps.

**Expected Structure for Rigasche Zeitung:**
- Base directory: `data/`
- Corpus subdirectory: `Rigasche_Zeitung_1918_1919/`
- Individual files: `rzei[year]s[volume]n[issue]_[page]_plaintext_s[segment].txt`
- Expected count: ~4,597 text files

In [5]:

from pathlib import Path

# Set up the path to our extracted data
extract_dir = Path("data")  # Relative path to the extraction directory

print("🔍 Verifying extracted Rigasche Zeitung corpus...")
print("=" * 60)

# Check if the extraction directory exists
if extract_dir.exists():
    # List all items in the extraction directory
    items = list(extract_dir.glob("*"))
    print(f"📁 Found {len(items)} items in extraction directory: {extract_dir}")
    print("\n📋 Contents of extraction directory:")
    print("-" * 40)
    
    for item in sorted(items):
        if item.is_dir():
            # Count files in subdirectory to give size indication
            subfiles = list(item.rglob("*"))
            file_count = len([f for f in subfiles if f.is_file()])
            print(f"  📁 {item.name}/ (contains {file_count} files)")
        else:
            # Show file size for individual files
            file_size = item.stat().st_size
            print(f"  📄 {item.name} ({file_size:,} bytes)")
    
    print(f"\n✅ Extraction directory verified successfully")
    
    # Check if we have the expected Rigasche Zeitung directory
    expected_corpus_dir = extract_dir / "Rigasche_Zeitung_1918_1919"
    if expected_corpus_dir.exists():
        print(f"🎯 Found expected corpus directory: {expected_corpus_dir.name}")
    else:
        print("⚠️  Expected 'Rigasche_Zeitung_1918_1919' directory not found")
        print("📝 Available directories:")
        for item in items:
            if item.is_dir():
                print(f"    - {item.name}")
    
else:
    print(f"❌ Extraction directory '{extract_dir}' does not exist!")
    print("🔧 Please verify that the extraction process completed successfully.")
    print("💡 Try running the extraction cell again if needed.")

🔍 Verifying extracted Rigasche Zeitung corpus...
📁 Found 2 items in extraction directory: data

📋 Contents of extraction directory:
----------------------------------------
  📁 Latvian_Economic_Review/ (contains 419 files)
  📁 Rigasche_Zeitung_1918_1919/ (contains 4597 files)

✅ Extraction directory verified successfully
🎯 Found expected corpus directory: Rigasche_Zeitung_1918_1919



### Comprehensive Analysis of Historical Corpus Structure

Now that we've confirmed the basic extraction, let's perform a detailed analysis of our Rigasche Zeitung corpus. This comprehensive analysis will help us understand:

#### **Corpus Composition:**
- **Total file count** and distribution by type
- **Text file inventory** (our primary working material)
- **Directory structure** and organization
- **File size distribution** to identify potential outliers

#### **Quality Assessment:**
- **File naming pattern verification** to ensure consistency
- **Sample content inspection** to check encoding and readability
- **Size analysis** to identify unusually small/large files that might indicate OCR issues

#### **Research Planning:**
- **Workload estimation** based on total file count and sizes
- **Sampling strategy** for initial testing and development
- **Processing priority** based on file characteristics

This analysis is particularly important for historical corpora like Rigasche Zeitung because:
- **OCR quality varies** across different issues and pages
- **Historical printing variations** affect digitization success
- **File size anomalies** often indicate OCR problems worth investigating
- **Systematic overview** helps plan computational resource allocation

The analysis function below provides both summary statistics and detailed file listings to support informed research decisions.

In [6]:

def analyze_directory_contents(directory_path, verbose=True):
    """
    Comprehensively analyze the contents of a directory recursively.
    Particularly useful for historical corpus analysis.
    
    Args:
        directory_path: Path object or string path to the directory
        verbose: If True, print detailed information about file types and structure
    
    Returns:
        dict: Dictionary containing comprehensive analysis results
    """
    from pathlib import Path
    
    directory = Path(directory_path)
    
    if not directory.exists():
        print(f"❌ Directory {directory} does not exist.")
        return None
    
    print(f"🔍 Analyzing corpus structure...")
    
    # Get all items recursively
    all_items = list(directory.rglob("*"))
    
    # Separate files from directories
    files_only = [f for f in all_items if f.is_file()]
    directories_only = [f for f in all_items if f.is_dir()]
    
    # Analyze file extensions
    file_extensions = {}
    total_size = 0
    for file in files_only:
        ext = file.suffix.lower()
        if ext == '':
            ext = '(no extension)'
        file_extensions[ext] = file_extensions.get(ext, 0) + 1
        total_size += file.stat().st_size
    
    # Focus on .txt files for text analysis
    txt_files = [f for f in files_only if f.suffix.lower() == '.txt']
    
    # Calculate text file statistics
    txt_sizes = [f.stat().st_size for f in txt_files] if txt_files else []
    
    # Compile analysis results
    results = {
        'total_items': len(all_items),
        'total_files': len(files_only),
        'total_directories': len(directories_only),
        'total_size_bytes': total_size,
        'txt_files_count': len(txt_files),
        'txt_files_paths': txt_files,
        'file_extensions': file_extensions,
        'txt_file_sizes': txt_sizes
    }
    
    if verbose:
        print(f"📊 CORPUS ANALYSIS REPORT")
        print("=" * 70)
        print(f"📁 Directory: {directory}")
        print(f"🔢 Total items (files + directories): {results['total_items']:,}")
        print(f"📄 Total files: {results['total_files']:,}")
        print(f"📁 Total subdirectories: {results['total_directories']:,}")
        print(f"💾 Total corpus size: {results['total_size_bytes']:,} bytes ({results['total_size_bytes']/1024/1024:.1f} MB)")
        print(f"📝 Text files (.txt): {results['txt_files_count']:,}")
        print()
        
        # File extension breakdown
        print("📋 FILE TYPE DISTRIBUTION:")
        print("-" * 40)
        for ext, count in sorted(file_extensions.items(), key=lambda x: x[1], reverse=True):
            percentage = (count / len(files_only)) * 100 if files_only else 0
            print(f"  {ext:15} {count:5,} files ({percentage:5.1f}%)")
        print()
        
        # Directory structure
        if directories_only:
            print("🗂️  DIRECTORY STRUCTURE:")
            print("-" * 40)
            for dir_path in sorted(directories_only):
                relative_path = dir_path.relative_to(directory_path)
                file_count = len([f for f in dir_path.rglob("*") if f.is_file()])
                print(f"  📁 {relative_path} ({file_count:,} files)")
            print()
        
        # Text file analysis
        if txt_files:
            if txt_sizes:
                avg_size = sum(txt_sizes) / len(txt_sizes)
                min_size = min(txt_sizes)
                max_size = max(txt_sizes)
                
                print("📝 TEXT FILE STATISTICS:")
                print("-" * 40)
                print(f"  Total text files: {len(txt_files):,}")
                print(f"  Average file size: {avg_size:,.0f} bytes")
                print(f"  Smallest file: {min_size:,} bytes")
                print(f"  Largest file: {max_size:,} bytes")
                print(f"  Total text content: {sum(txt_sizes):,} bytes ({sum(txt_sizes)/1024/1024:.1f} MB)")
                print()
            
            # Sample file listing
            sample_count = min(10, len(txt_files))
            print(f"📋 SAMPLE TEXT FILES (showing {sample_count} of {len(txt_files)}):")
            print("-" * 70)
            for txt_file in sorted(txt_files)[:sample_count]:
                relative_path = txt_file.relative_to(directory_path)
                file_size = txt_file.stat().st_size
                print(f"  📄 {str(relative_path):50} {file_size:6,} bytes")
            
            if len(txt_files) > sample_count:
                print(f"  ... and {len(txt_files) - sample_count:,} more text files")
            print()
    
    return results

# Perform comprehensive analysis of the extracted Rigasche Zeitung corpus
print("🚀 Starting comprehensive corpus analysis...")
print("⏱️  This may take a moment for large corpora...")
print()

analysis_results = analyze_directory_contents(extract_dir, verbose=True)

if analysis_results:
    print("✅ Corpus analysis completed successfully!")
    print(f"🎯 Ready to work with {analysis_results['txt_files_count']:,} historical text files")
    
    # Quick validation against expected corpus size
    expected_files = 4597  # Expected number of segments for Rigasche Zeitung
    actual_files = analysis_results['txt_files_count']
    
    if actual_files == expected_files:
        print(f"✅ File count matches expected corpus size ({expected_files:,} files)")
    elif actual_files > 0:
        print(f"⚠️  File count ({actual_files:,}) differs from expected ({expected_files:,})")
        print("   This might be normal depending on the corpus version or processing")
    else:
        print(f"❌ No text files found! Please check the extraction process.")
else:
    print("❌ Corpus analysis failed. Please check the extraction directory.")

🚀 Starting comprehensive corpus analysis...
⏱️  This may take a moment for large corpora...

🔍 Analyzing corpus structure...
📊 CORPUS ANALYSIS REPORT
📁 Directory: data
🔢 Total items (files + directories): 5,018
📄 Total files: 5,016
📁 Total subdirectories: 2
💾 Total corpus size: 33,554,554 bytes (32.0 MB)
📝 Text files (.txt): 5,016

📋 FILE TYPE DISTRIBUTION:
----------------------------------------
  .txt            5,016 files (100.0%)

🗂️  DIRECTORY STRUCTURE:
----------------------------------------
  📁 Latvian_Economic_Review (419 files)
  📁 Rigasche_Zeitung_1918_1919 (4,597 files)

📝 TEXT FILE STATISTICS:
----------------------------------------
  Total text files: 5,016
  Average file size: 6,690 bytes
  Smallest file: 85 bytes
  Largest file: 55,914 bytes
  Total text content: 33,554,554 bytes (32.0 MB)

📋 SAMPLE TEXT FILES (showing 10 of 5016):
----------------------------------------------------------------------
  📄 Latvian_Economic_Review\lerq1936s01n01_003_plaintext_s01.txt 

In [8]:
analysis_results.keys()

dict_keys(['total_items', 'total_files', 'total_directories', 'total_size_bytes', 'txt_files_count', 'txt_files_paths', 'file_extensions', 'txt_file_sizes'])

In [10]:

# Select the 6th text file for examination (good size for sample analysis)
# We choose the 6th file as it's likely to be representative but not too large for display

print("🔍 SELECTING SAMPLE DOCUMENT FOR ANALYSIS")
print("=" * 50)

text_files_list = analysis_results['txt_files_paths']

if len(text_files_list) >= 6:
    sixth_text_file = text_files_list[-6]  # 6th file from end (from start would be text_files_list[5])
    file_size = sixth_text_file.stat().st_size
    
    print(f"📄 Selected file: {sixth_text_file.name}")
    print(f"📊 File size: {file_size:,} bytes")
    print(f"📁 Full path: {sixth_text_file}")
    print()
    
    # Read and display the file contents
    try:
        print("📖 DOCUMENT CONTENT PREVIEW:")
        print("-" * 70)
        
        with sixth_text_file.open('r', encoding='utf-8') as f:
            content = f.read()
        
        # Display file statistics
        word_count = len(content.split())
        line_count = len(content.splitlines())
        char_count = len(content)
        
        print(f"📊 Content statistics:")
        print(f"   Characters: {char_count:,}")
        print(f"   Words (approx): {word_count:,}")
        print(f"   Lines: {line_count:,}")
        print()
        
        # Show first 500 characters to preview content and potential OCR issues
        preview_length = 500
        print(f"🔍 Content preview (first {preview_length} characters):")
        print("─" * 70)
        print(content[:preview_length])
        
        if len(content) > preview_length:
            print(f"\n... (document continues for {len(content) - preview_length:,} more characters)")
        
        print("─" * 70)
        
        # Quick OCR quality assessment
        print("\n🔍 PRELIMINARY OCR QUALITY ASSESSMENT:")
        print("-" * 50)
        
        # Check for common OCR error indicators
        ocr_indicators = {
            'scattered_digits': len([c for c in content if c.isdigit()]) / len(content) if content else 0,
            'unusual_punctuation': content.count('~') + content.count('|') + content.count('#'),
            'repeated_spaces': content.count('  '),
            'mixed_case_words': len([w for w in content.split() if w and any(c.isupper() for c in w) and any(c.islower() for c in w)]),
        }
        
        print(f"📈 OCR Quality Indicators:")
        print(f"   Digit density: {ocr_indicators['scattered_digits']:.3f} (lower is usually better)")
        print(f"   Unusual punctuation marks: {ocr_indicators['unusual_punctuation']}")
        print(f"   Multiple spaces: {ocr_indicators['repeated_spaces']}")
        print(f"   Mixed case words: {ocr_indicators['mixed_case_words']}")
        
        # Assess overall quality
        if ocr_indicators['scattered_digits'] < 0.05 and ocr_indicators['unusual_punctuation'] < 10:
            print("✅ OCR quality appears relatively good")
        elif ocr_indicators['scattered_digits'] < 0.1 and ocr_indicators['unusual_punctuation'] < 25:
            print("⚠️  OCR quality appears moderate - some errors expected")
        else:
            print("🔧 OCR quality appears challenging - significant errors likely")
        
        print(f"\n🎯 This document will be excellent for testing LLM-based OCR correction!")
        
    except UnicodeDecodeError as e:
        print(f"❌ Encoding error reading file: {e}")
        print("🔧 This indicates potential character encoding issues in the historical text")
    except Exception as e:
        print(f"❌ Error reading file: {e}")
        
else:
    print(f"❌ Insufficient files in corpus. Found {len(text_files_list)} files, need at least 6.")
    if len(text_files_list) > 0:
        print("📋 Available files:")
        for i, file_path in enumerate(text_files_list[:5]):
            print(f"   {i+1}. {file_path.name}")

🔍 SELECTING SAMPLE DOCUMENT FOR ANALYSIS
📄 Selected file: rzei1919s01n60_002_plaintext_s05.txt
📊 File size: 23,852 bytes
📁 Full path: data\Rigasche_Zeitung_1918_1919\rzei1919s01n60_002_plaintext_s05.txt

📖 DOCUMENT CONTENT PREVIEW:
----------------------------------------------------------------------
📊 Content statistics:
   Characters: 22,823
   Words (approx): 3,190
   Lines: 565

🔍 Content preview (first 500 characters):
──────────────────────────────────────────────────────────────────────
title: Ausland.
author: 
uri: http://dom.lndb.lv/data/obj/318311



Deutsches Reich.
Annahme der Verfassung.
Die Nationalversammlung nahm am 31. Juli
die Versassung des neuen Deutschen Reichs in
dritter Lesung durch namentliche Abstimmung

endgültig an. Die Mehrheit des Parlaments,
aus Sozialdemokraten, Zentrum uud der
Deutschen demokratischen Partei bestehend, gab
262 Stimmen dafür ab. Dagegen stimmten die
Deutschnationale Volkspartei, die Deutsche-Volks-
Partei und die Unabhängige sozialdemokr